In [8]:
import numpy as np
import nibabel as nib
#import matplotlib.pyplot as plt
import os

In [9]:
os.getcwd()

'C:\\Users\\David\\Desktop\\Brain_Research\\Highres_Tracking'

# Terminologies

aseg does Subcortical segmentation: White matter, Cortex(not used) Caudate, Pallidum, Hippocampus, Putamen, Thalamus, Lateral Ventricle, Cerebellium, Nuclelus, Accumbens, etc.

aparc creates parcels of the cortex (surface only)

# 100307 3T

In [2]:
Diff_dir="C:\\Users\\David\\Desktop\\host\\100307_3T_Diffusion_preproc"
Struct_dir="C:\\Users\\David\\Desktop\\host\\100307_3T_Structural_preproc"
Structex_dir="C:\\Users\\David\\Desktop\\100307_3T_Structural_preproc_extended"

In [10]:
import nibabel as nib
from dipy.tracking.eudx import EuDX
from dipy.reconst import peaks, shm
from dipy.tracking import utils
from dipy.core import gradients


data_img=nib.load(Diff_dir+"\\100307\\T1w\\Diffusion\\data.nii")
data=data_img.get_data()
data_affine=data_img.affine

Atlas_labels_img=nib.load(Struct_dir+"\\MNINonLinear\\ROIs\\wmparcs.2.nii.gz")
Atlas_labels=Atlas_labels_img.get_data()

labels_img=nib.load(Struct_dir+"\\T1w\\aparc+aseg.nii.gz")
labels=labels_img.get_data()


wmparcs_img=nib.load(Struct_dir+"\\T1w\\wmparc.nii.gz")
wmparcs=wmparcs_img.get_data()

t1_img=nib.load(Diff_dir+"\\100307\\T1w\\T1w_acpc_dc_restore_1.25.nii.gz")
t1_data=t1_img.get_data()

bvals_file=Diff_dir+"\\100307\\T1w\\Diffusion\\bvals_T.txt"
bvals=np.genfromtxt(bvals_file)

bvecs_file=Diff_dir+"\\100307\\T1w\\Diffusion\\bvecs_T.txt"
bvecs=np.genfromtxt(bvecs_file)

gtab=gradients.gradient_table(bvals=bvals,bvecs=bvecs)

m_img=Diff_dir+"\\100307\\T1w\\Diffusion\\nodif_brain_mask.nii.gz"
m=nib.load(m_img)
mm=m.get_data()

NameError: name 'Diff_dir' is not defined

# 104012 3T

# Deterministic Maximum Direction Getter

create appropriate setup for loading data

In [11]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import os
import time
import pickle
import sys
import random
import collections

In [12]:
Diff_dir="C:\\Users\\David\\Desktop\\host\\104012_3T_Diffusion_preproc"
Struct_dir="C:\\Users\\David\\Desktop\\host\\104012_3T_Structural_preproc"
Structex_dir="C:\\Users\\David\\Desktop\\104012_3T_Structural_preproc_extended"

In [13]:
import nibabel as nib
from dipy.tracking.eudx import EuDX
from dipy.reconst import peaks, shm
from dipy.tracking import utils
from dipy.core import gradients

from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel
from dipy.tracking import utils
from dipy.tracking.local import (ThresholdTissueClassifier, LocalTracking)

# DATA
data_img=nib.load(Diff_dir+"\\104012\\T1w\\Diffusion\\data.nii\\data.nii")
data=data_img.get_data()
data_affine=data_img.affine

#ATLAS. POOR ROI
Atlas_labels_img=nib.load(Struct_dir+"\\104012\\MNINonlinear\\ROIs\\Atlas_wmparc.2.nii.gz")
Atlas_labels=Atlas_labels_img.get_data()

# LOADING REGION OF INTERESTS
labels_img=nib.load(Struct_dir+"\\104012\\T1w\\aparc+aseg.nii.gz")
labels=labels_img.get_data()
labels_affine=labels_img.affine

# WHITE MATTER PARCELS. BUT SOMEHOW INCLUDES EVERYTHING
wmparcs_img=nib.load(Struct_dir+"\\104012\\T1w\\wmparc.nii.gz")
wmparcs=wmparcs_img.get_data()

# REAL WHITE MATTER

# T1 IMAGE NECESSARY FOR TRACKTOGRAPHY
t1_img=nib.load(Diff_dir+"\\104012\\T1w\\T1w_acpc_dc_restore_1.25.nii.gz")
t1_data=t1_img.get_data()

# BVALS AND BVECS
bvals_file=Diff_dir+"\\104012\\T1w\\Diffusion\\bvals_T.txt"
bvals=np.genfromtxt(bvals_file)
bvecs_file=Diff_dir+"\\104012\\T1w\\Diffusion\\bvecs_T.txt"
bvecs=np.genfromtxt(bvecs_file)





Need a gradient table for tractography

In [4]:
gtab=gradients.gradient_table(bvals=bvals,bvecs=bvecs)

Below are 1,0 map of brain_mask and its boolean (True and False) version.

In [5]:
m_img=Diff_dir+"\\104012\\T1w\\Diffusion\\nodif_brain_mask.nii.gz"
m=nib.load(m_img)
mm=m.get_data()
mask_bool=(mm==1)

seed mask of CC and some white matter, using emparcs

In [6]:
labels.shape

(260, 311, 260)

# HT import forward station

In [ ]:
del sys.modules['HTracking1_tools_alt8']

In [6]:
import HTracking1_tools_alt9 as HT

In [765]:
'''superlist=[]
for i in range(labels.shape[0]):
    for j in range(labels.shape[1]):
        for k in range(labels.shape[2]):
            if labels[i][j][k] not in superlist and labels[i][j][k]!=0:
                superlist.append(labels[i][j][k])'''
#outsource the work to cython
superlist=HT.make_superlist(labels)

KeyboardInterrupt: 

In [7]:
#superlist.sort()
for p in [16,2,7,41,46]:
    print("(labels==",int(p),")|",end="")

(labels== 16 )|(labels== 2 )|(labels== 7 )|(labels== 41 )|(labels== 46 )|

In [8]:
seed_mask=(labels== 16 )|(labels== 2 )|(labels== 7 )|(labels== 41 )|(labels== 46 )
seed_mask.shape

(260, 311, 260)

create seeds. They consist of coordinates (x,y,z) in the REFERENCE SPACE not in voxel space.

In [9]:
seeds = utils.seeds_from_mask(seed_mask, density=1, affine=labels_img.affine)
print(seeds.shape)

(1323769, 3)


# CSD FIT

csd: Constrainted spherical deconvolution.  
It estimates response function (relationship between input and output) of the data then it deconvolves the measured signals to obtain the fiber orientation distribution  (FOD).  

Since the diffusion data have different dimension as the labels, the csd fit must be performed on ALL data, including gray matter up to the mask.

In [10]:
csd_model = ConstrainedSphericalDeconvModel(gtab, None, sh_order=6)

In [11]:
csd_fit = csd_model.fit(data, mask=mask_bool) # COMPUTATIONALLY INTENSE

# CSD FIT SAVE

computation of csd_fit is quite intense and it would be foolish to compute it everytime I open this notebook. Thus I save it as a file to be loaded when in need.

In [12]:
'''with open('csd_fit_file.pkl', 'wb') as output:
    pickle.dump(csd_fit, output, pickle.HIGHEST_PROTOCOL)'''

"with open('csd_fit_file.pkl', 'wb') as output:\n    pickle.dump(csd_fit, output, pickle.HIGHEST_PROTOCOL)"

# CSD FIT LOAD

In [13]:
'''sys.setrecursionlimit(9000)
try:
    with open('csd_fit_file.pkl', 'rb') as input:
        csd_fit_load=pickle.load(input)
except:
    print("failed")'''

'sys.setrecursionlimit(9000)\ntry:\n    with open(\'csd_fit_file.pkl\', \'rb\') as input:\n        csd_fit_load=pickle.load(input)\nexcept:\n    print("failed")'

# Tissue Classifier
Use FA as a tissue classifier that tells when to stop tracking. Again, I cannot exclusively select white matter in the resolution of data, so every point upto the mask is considered.

In [14]:
import dipy.reconst.dti as dti
from dipy.reconst.dti import fractional_anisotropy

tensor_model = dti.TensorModel(gtab)

In [15]:
tenfit=tensor_model.fit(data,mask=mask_bool) #COMPUTATIONALL INTENSE

In [66]:
FA=fractional_anisotropy(tenfit.evals)

In [67]:
classifier=ThresholdTissueClassifier(FA,.05) # 0.2 enough?

# Streamlines
Use the csd fit model that estimated the FOD to generate streamlines

In [68]:
from dipy.data import default_sphere
from dipy.direction import DeterministicMaximumDirectionGetter
from dipy.io.trackvis import save_trk
import time

In [69]:
"""start=time.time()
detmax_dg = DeterministicMaximumDirectionGetter.from_shcoeff(csd_fit.shm_coeff,
                                                             max_angle=30.,
                                                             sphere=default_sphere)
end=time.time()
print("time taken: ",end-start," seconds")
print("time taken: ",(end-start)/60," minutess")"""

'start=time.time()\ndetmax_dg = DeterministicMaximumDirectionGetter.from_shcoeff(csd_fit.shm_coeff,\n                                                             max_angle=30.,\n                                                             sphere=default_sphere)\nend=time.time()\nprint("time taken: ",end-start," seconds")\nprint("time taken: ",(end-start)/60," minutess")'

In [70]:
start=time.time()
from dipy.direction import ProbabilisticDirectionGetter

prob_dg = ProbabilisticDirectionGetter.from_shcoeff(csd_fit.shm_coeff,
                                                    max_angle=35.,
                                                    sphere=default_sphere)
end=time.time()
print("time taken: ",end-start," seconds")
print("time taken: ",(end-start)/60," minutess")

time taken:  4.024363994598389  seconds
time taken:  0.06707273324330648  minutess


In [71]:
streamlines = LocalTracking(prob_dg, classifier,
                            seeds,data_img.affine ,
                            step_size=0.7/2)


# Unused connectivity Matrix

In [72]:
"""start=time.time()
M, grouping = utils.connectivity_matrix(streamlines, intlabels,affine=data_img.affine,
                                        return_mapping=True,
                                        mapping_as_streamlines=True)
end=time.time()
print("time taken: ",end-start," seconds")
print("time taken: ",(end-start)/60," minutess")"""

'start=time.time()\nM, grouping = utils.connectivity_matrix(streamlines, intlabels,affine=data_img.affine,\n                                        return_mapping=True,\n                                        mapping_as_streamlines=True)\nend=time.time()\nprint("time taken: ",end-start," seconds")\nprint("time taken: ",(end-start)/60," minutess")'

# HT forward station

In [988]:
del sys.modules['HTracking1_tools_alt40']

In [29]:
import HTracking1_tools_alt44 as HT

# Custom Connectivity
## Streamlines

In [73]:
1

1

In [ ]:
begin=time.time()
liststreamlines=list(streamlines)
end=time.time()
print(end-begin," seconds")
print((end-begin)/60," minutes")

In [24]:
begin=time.time()
astreamlines=np.array(liststreamlines)
end=time.time()
print(end-begin)

63.50283646583557


In [25]:
begin=time.time()
endpoints=np.array([st[0::len(st)-1] for st in astreamlines if len(st)>1] )
end=time.time()
print(end-begin)

4354.7477169036865


In [26]:
len(liststreamlines)

2981117

In [27]:
len(endpoints)

2981117

In [30]:
endpointsVOX=HT.map_ep_ref_to_voxel(endpoints,labels_img.affine)
endpointsVOX_INT=np.rint(endpointsVOX)
endpointsVOX_INT=endpointsVOX_INT.astype('int16')

In [31]:
randlist=[]
for i in range(30):
    a=random.randint(0,128000)
    b=random.randint(0,1)
    c=random.randint(0,2)
    assert (endpointsVOX[a][b][c]-endpointsVOX_INT[a][b][c])<0.5

# Forward station

In [73]:
del sys.modules['HTracking1_tools_alt43']

KeyError: 'HTracking1_tools_alt43'

In [74]:
import HTracking1_tools_alt44 as HT

## ROI

In [14]:
roi_index_list=[]
for i in range(0,labels.shape[0]):
    for j in range(0,labels.shape[1]):
        for k in range(0,labels.shape[2]):
            if labels[i][j][k] not in roi_index_list:
                roi_index_list.append(int(labels[i][j][k]))

roi_index_list.sort()

In [33]:
roi_dict_TF={} #True/False table for the roi dictonary
roi_dict_vox={} #voxel coordinate dictionary
roi_dict_ref={} #reference coordinate dictionary

for i in roi_index_list:
    roi_dict_TF[i]=labels==int(i) # link rei index to roi true/false table in voxel coord
    
    where=np.where(roi_dict_TF[i])
    wherearray=np.array([where[0],where[1],where[2]])
    roi_dict_vox[i]=wherearray.T.astype('int16')
    
    roi_dict_ref[i]=HT.map_voxel_to_ref(where,labels_img.affine)

In [34]:
assert 260*311*260==sum([roi_dict_vox[i].shape[0] for i in roi_index_list])

In [35]:
sum([roi_dict_vox[i].shape[0] for i in roi_index_list])

21023600

eliminate 0,2,7,41,46

In [36]:
roi_index_list_nowhite=[]
for i in roi_index_list:
    if i not in [0.0,2.0,7.0,41.0,46.0,251.0,252.0,253.0,254.0,255.0]:
        roi_index_list_nowhite.append(i)

In [37]:
sum([roi_dict_vox[i].shape[0] for i in roi_index_list_nowhite])

2016000

# Checking Station

In [2]:
import numpy as np

In [38]:
endpointsVOX_INT.shape

(2981117, 2, 3)

In [39]:
endpointsVOX_INT.dtype

dtype('int16')

In [40]:
endpointsVOX_INT

array([[[ 34, 141,  95],
        [ 50, 121, 108]],

       [[ 37, 129,  99],
        [ 53, 141, 100]],

       [[ 46, 130,  93],
        [ 43, 137, 102]],

       ..., 
       [[204, 128,  90],
        [213, 140,  97]],

       [[205, 151,  86],
        [217, 136,  97]],

       [[191, 126, 102],
        [222, 143,  92]]], dtype=int16)

In [41]:
for  i in roi_index_list_nowhite:
    assert roi_dict_vox[i].dtype==np.dtype('int16')

In [42]:
print(len(roi_index_list))
print(len(roi_index_list_nowhite))

113
103


In [43]:
t={'[1,16]':2}
t['[1,16]']

2

# Better Analyze

In [44]:
i,j,k=endpointsVOX_INT.T

In [45]:
labels_int_ep=labels[i,j,k].astype('int16')

In [46]:
begin=labels_int_ep[0]
end=labels_int_ep[1]
begin

array([   0, 2015, 2015, ..., 1015, 1015, 1001], dtype=int16)

In [47]:
end

array([2008, 2015, 2015, ..., 1030, 1015,    0], dtype=int16)

In [48]:
raw_dictionary=HT.connectivity_matrix(begin,end,roi_index_list)

In [49]:
sym_dictionary=HT.make_2d_dictionary_symmetric(roi_index_list)

# temporary calculation cell for the interim 

In [50]:
len(raw_dictionary)

113

In [51]:
for i in roi_index_list:
    for j in roi_index_list:
        sym_dictionary[i][j]=raw_dictionary[i][j]+raw_dictionary[j][i]

In [52]:
for i in roi_index_list:
    sym_dictionary[16][i]

In [53]:
for i in roi_index_list:
    print(sym_dictionary[16][i])
    assert sym_dictionary[16][i]==raw_dictionary[16][i]+raw_dictionary[i][16]

12082
229
210
24
13
408
22
138
67
100
99
448
2596
178
31
374
1
25
0
85
548
380
14
55
517
25
74
11
10
660
22
1
66
0
118
2
0
1
1
0
4
0
0
1
9
90
82
4
32
306
45
42
9
129
20
34
40
56
163
29
82
3
57
7
36
63
115
20
16
144
570
24
83
181
1
0
5
145
34
62
45
72
4
218
373
205
267
11
57
22
188
18
448
115
105
29
21
21
1
197
67
551
82
3
16
355
34
64
320
0
2
27
115


In [48]:
roi_list_temp=list(np.arange(2000,2004))+list(np.arange(1000,1004))+list(np.arange(1005,1036))+list(np.arange(2005,2036))+list(np.arange(10,19))+[0,2,4,5,7,8,24,26,28,30,31,41,43,44,47,49,50,51,52,53,54,58,60,62,63,77,80,85]

## 1

In [52]:
for i in roi_list_temp:
    print("number",i," : ",loaded[i][16])+
    

number 2000  :  5
number 2001  :  216
number 2002  :  171
number 2003  :  162
number 1000  :  16
number 1001  :  15
number 1002  :  30
number 1003  :  113
number 1005  :  0
number 1006  :  3
number 1007  :  76
number 1008  :  198
number 1009  :  17
number 1010  :  4
number 1011  :  1
number 1012  :  13
number 1013  :  1
number 1014  :  15
number 1015  :  35
number 1016  :  53
number 1017  :  23
number 1018  :  311
number 1019  :  1
number 1020  :  167
number 1021  :  4
number 1022  :  26
number 1023  :  35
number 1024  :  125
number 1025  :  11
number 1026  :  6
number 1027  :  305
number 1028  :  733
number 1029  :  17
number 1030  :  97
number 1031  :  482
number 1032  :  0
number 1033  :  0
number 1034  :  6
number 1035  :  372
number 2005  :  2
number 2006  :  141
number 2007  :  131
number 2008  :  279
number 2009  :  101
number 2010  :  3
number 2011  :  14
number 2012  :  1
number 2013  :  10
number 2014  :  1
number 2015  :  109
number 2016  :  60
number 2017  :  251
number 201

## 4

In [54]:
for i in roi_list_temp:
    print("number",i," : ",loaded[i][16])

number 2000  :  33
number 2001  :  25
number 2002  :  10
number 2003  :  8
number 1000  :  1
number 1001  :  4
number 1002  :  24
number 1003  :  13
number 1005  :  1
number 1006  :  14
number 1007  :  31
number 1008  :  3
number 1009  :  18
number 1010  :  4
number 1011  :  5
number 1012  :  12
number 1013  :  3
number 1014  :  32
number 1015  :  30
number 1016  :  95
number 1017  :  2
number 1018  :  35
number 1019  :  1
number 1020  :  33
number 1021  :  1
number 1022  :  4
number 1023  :  13
number 1024  :  20
number 1025  :  4
number 1026  :  11
number 1027  :  39
number 1028  :  102
number 1029  :  5
number 1030  :  40
number 1031  :  10
number 1032  :  1
number 1033  :  0
number 1034  :  4
number 1035  :  56
number 2005  :  1
number 2006  :  218
number 2007  :  218
number 2008  :  102
number 2009  :  215
number 2010  :  7
number 2011  :  34
number 2012  :  18
number 2013  :  101
number 2014  :  13
number 2015  :  410
number 2016  :  66
number 2017  :  17
number 2018  :  19
numbe

# Save & load Station

In [ ]:
with open('dictionary0','wb') as f:
    pickle.dump(cdict)

In [54]:
with open('dictionary4','wb') as f:
    pickle.dump(raw_dictionary,f,pickle.HIGHEST_PROTOCOL)

In [5]:
with open ('dictionary4','rb' ) as f:
    loaded=pickle.load(f)

In [6]:
loaded

{0: {0: 360443,
  2: 11369,
  4: 12768,
  5: 632,
  7: 507,
  8: 11431,
  10: 485,
  11: 3434,
  12: 1458,
  13: 1641,
  14: 965,
  15: 6787,
  16: 6436,
  17: 4456,
  18: 688,
  24: 1817,
  26: 33,
  28: 418,
  30: 0,
  31: 3208,
  41: 11098,
  43: 14111,
  44: 555,
  46: 937,
  47: 10195,
  49: 324,
  50: 3288,
  51: 632,
  52: 551,
  53: 2274,
  54: 349,
  58: 120,
  60: 705,
  62: 1,
  63: 3424,
  77: 323,
  80: 0,
  85: 44,
  251: 11,
  252: 6,
  253: 14,
  254: 7,
  255: 21,
  1000: 63,
  1001: 2891,
  1002: 3360,
  1003: 11181,
  1005: 3354,
  1006: 811,
  1007: 15540,
  1008: 10364,
  1009: 6771,
  1010: 1788,
  1011: 15345,
  1012: 4749,
  1013: 10008,
  1014: 2429,
  1015: 14114,
  1016: 2784,
  1017: 6328,
  1018: 5151,
  1019: 1227,
  1020: 3516,
  1021: 3408,
  1022: 10526,
  1023: 3866,
  1024: 16258,
  1025: 12261,
  1026: 1983,
  1027: 19735,
  1028: 33988,
  1029: 15525,
  1030: 11904,
  1031: 14266,
  1032: 184,
  1033: 588,
  1034: 1644,
  1035: 5420,
  2000: 151,
  

In [15]:
loaded[16]

{0: 5646,
 2: 145,
 4: 140,
 5: 15,
 7: 7,
 8: 194,
 10: 14,
 11: 98,
 12: 46,
 13: 64,
 14: 64,
 15: 171,
 16: 1298,
 17: 102,
 18: 16,
 24: 288,
 26: 0,
 28: 19,
 30: 0,
 31: 51,
 41: 92,
 43: 228,
 44: 2,
 46: 31,
 47: 238,
 49: 16,
 50: 51,
 51: 7,
 52: 4,
 53: 37,
 54: 6,
 58: 1,
 60: 43,
 62: 0,
 63: 74,
 77: 2,
 80: 0,
 85: 0,
 251: 1,
 252: 0,
 253: 3,
 254: 0,
 255: 0,
 1000: 0,
 1001: 5,
 1002: 66,
 1003: 69,
 1005: 3,
 1006: 18,
 1007: 275,
 1008: 42,
 1009: 24,
 1010: 5,
 1011: 124,
 1012: 8,
 1013: 31,
 1014: 8,
 1015: 26,
 1016: 68,
 1017: 27,
 1018: 47,
 1019: 2,
 1020: 24,
 1021: 6,
 1022: 32,
 1023: 50,
 1024: 95,
 1025: 16,
 1026: 5,
 1027: 105,
 1028: 468,
 1029: 19,
 1030: 43,
 1031: 171,
 1032: 0,
 1033: 0,
 1034: 1,
 1035: 89,
 2000: 1,
 2001: 37,
 2002: 35,
 2003: 64,
 2005: 3,
 2006: 0,
 2007: 155,
 2008: 103,
 2009: 52,
 2010: 4,
 2011: 23,
 2012: 4,
 2013: 87,
 2014: 5,
 2015: 38,
 2016: 49,
 2017: 88,
 2018: 10,
 2019: 2,
 2020: 2,
 2021: 0,
 2022: 150,
 2023

In [57]:
assert raw_dictionary==loaded

In [7]:
loaded[1011][16]

5

In [5]:
loaded[47][16]

279

# analysis

In [146]:
def apply(master,peasant,index_list):
    for i in index_list:
        master[i]+=peasant[i]
    return master

In [147]:
master_dictionary={}
for i in roi_index_list_nowhite:
    master_dictionary[i]=0

In [148]:
'''
iter_list=[]
for i in range(0,128700,2000):
    for j in range(0,375):
        iter_list.append(i+j)
len(iter_list)
p=endpointsVOX_INT[86500:86600]
'''

'\niter_list=[]\nfor i in range(0,128700,2000):\n    for j in range(0,375):\n        iter_list.append(i+j)\nlen(iter_list)\np=endpointsVOX_INT[86500:86600]\n'

In [149]:
TB=time.time()
total_time=[]
total_count=[]
total_dictionary=[]

for i in range(0,126700,2000): #2000 before
    begin=time.time()
    
    
    count_from,returned_dict=HT.single_connectivity(endpointsVOX_INT[i:i+500],roi_dict_vox[16],roi_dict_vox, roi_index_list_nowhite)
    master_dictionary=apply(master_dictionary,returned_dict,roi_index_list_nowhite)
    total_count.append(count_from)
    total_dictionary.append([returned_dict,master_dictionary])
    
    
    end=time.time()
    total_time.append(end-begin)
TE=time.time()
print(TE-TB, " seconds")
print((TE-TB)/60," minutes")

17172.749398469925  seconds
286.21248997449874  minutes


In [170]:
for i in roi_index_list:
    if i not in roi_index_list_nowhite:
        print('NA')
    else:
        print(master_dictionary[i])

NA
NA
0
4
NA
565
184
17
51
24
99
409
14830
88
11
5
0
575
1
5
NA
8
1
NA
578
101
29
15
2
18
5
0
563
0
12
6
0
0
NA
NA
NA
NA
NA
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
3
0
2
0
0
0
0
0
1
0
0
2
6
0
1
2
0
0
0
17
0
1
0
2
0
0
1
1
2
0
0
0
0
0
1
2
5
2
0
0
0
10
0
12
0
0
0
4
0
0
8
0
0
3
2


In [142]:
id(save_fict)

2172916173768

In [143]:
id(master_dictionary)

2172916173768

In [ ]:
1.44 default
1.96 all range
1.34 hybrid

In [816]:
list1

[[4, 767, 0],
 [5, 767, 0],
 [8, 767, 0],
 [10, 767, 0],
 [11, 767, 0],
 [12, 767, 0],
 [13, 767, 0],
 [14, 767, 0],
 [15, 767, 4],
 [16, 767, 394]]

In [817]:
TB=time.time()
list1=[]
for i in roi_index_list_nowhite[10:30]:
    a1,a2=HT.single_connectivity(endpoints_short,roi_dict_vox[16],roi_dict_vox[i])
    list1.append([i,a1,a2])
    print(i,a1,a2)

TE=time.time()
print(TE-TB, " seconds")
print((TE-TB)/60," minutes")

17 767 0
18 767 0
24 767 0
26 767 0
28 767 0
30 767 0
31 767 0
43 767 0
44 767 0
47 767 37
49 767 1
50 767 0
51 767 0
52 767 0
53 767 0
54 767 0
58 767 0
60 767 50
62 767 0
63 767 0
488.7181832790375  seconds
8.145303054650624  minutes


In [819]:
TB=time.time()
list1=[]
for i in roi_index_list_nowhite[30:109]:
    a1,a2=HT.single_connectivity(endpoints_short,roi_dict_vox[16],roi_dict_vox[i])
    list1.append([i,a1,a2])
    print(i,a1,a2)

TE=time.time()
print(TE-TB, " seconds")
print((TE-TB)/60," minutes")

77 767 0
80 767 0
85 767 0
251 767 0
252 767 0
253 767 0
254 767 0
255 767 0
1000 767 0
1001 767 0
1002 767 0
1003 767 0
1005 767 0
1006 767 0
1007 767 0
1008 767 0
1009 767 0
1010 767 0
1011 767 0
1012 767 0
1013 767 0
1014 767 0
1015 767 0
1016 767 0
1017 767 0
1018 767 0
1019 767 0
1020 767 0
1021 767 0
1022 767 0
1023 767 0
1024 767 0
1025 767 0
1026 767 0
1027 767 0
1028 767 0
1029 767 0
1030 767 0
1031 767 0
1032 767 0
1033 767 0
1034 767 0
1035 767 0
2000 767 0
2001 767 0
2002 767 0
2003 767 0
2005 767 0
2006 767 0
2007 767 0
2008 767 0
2009 767 0
2010 767 0
2011 767 0
2012 767 0
2013 767 0
2014 767 0
2015 767 0
2016 767 0
2017 767 0
2018 767 0
2019 767 0
2020 767 0
2021 767 0
2022 767 0
2023 767 0
2024 767 0
2025 767 0
2026 767 0
2027 767 0
2028 767 0
2029 767 0
2030 767 0
2031 767 0
2032 767 0
2033 767 0
2034 767 0
2035 767 0
1985.9307415485382  seconds
33.098845692475635  minutes


In [654]:
a=np.zeros([1000000,3])
b=np.zeros([10,3])
begin=time.time()
for i in a:
    for j in b:
        if i[0]==j[0]:
            if i[1]==j[1]:
                if i[2]==j[2]:
                    None
end=time.time()
print(end-begin)

9.311280965805054


In [656]:
a=np.zeros([1000000,3])
b=np.zeros([10,3])
begin=time.time()
for i in a:
    for j in b:
        c=i==j
        if c.any():
            None
end=time.time()
print(end-begin)

21.959826707839966


In [681]:
roi_dict_vox[16]

array([[105, 145,  93],
       [106, 145,  93],
       [107, 145,  91],
       ..., 
       [153, 149,  63],
       [153, 149,  64],
       [153, 149,  65]], dtype=int16)

In [649]:
np,where(105,roi_dict_vox[16])

TypeError: 'tuple' object is not callable

In [643]:
np.sort(endpoints[4],axis=0)

array([[ 15.29102293, -27.27292991,  -7.3684276 ],
       [ 60.91481749, -24.60732217,  17.04417812]])

In [633]:
connec

{16: 0}

In [550]:
TB=time.time()

connec=HT.connectivity(endpointsVOX_INT,roi_dict_vox,[4,5,7,80])




TE=time.time()
print(TE-TB, " seconds")
print((TE-TB)/60," minutes")

{80: {4: 0, 5: 0, 7: 0}, 4: {80: 0, 5: 0, 7: 0}, 5: {80: 0, 4: 0, 7: 0}, 7: {80: 0, 4: 0, 5: 0}}


KeyboardInterrupt: 

In [160]:
lis[9988] # lis of the beginning points time spend: 146 minutes

[1, 19113, 3456, 1, array([[  1.4368304 , -38.58460097, -38.36170113],
        [  8.39717781, -31.15000153, -12.61975864]]), array([  8.10000139, -30.80000162, -12.50000101])]

In [155]:

len(lis)

89626

In [512]:
if True:
    raise ValueError(':D')

ValueError: :D

In [530]:
for i in roi_index_list:
    print(i,roi_dict_vox[i].shape)

0 (17730658, 3)
2 (594345, 3)
4 (11087, 3)
5 (776, 3)
7 (32338, 3)
8 (152587, 3)
10 (23085, 3)
11 (10027, 3)
12 (15822, 3)
13 (4168, 3)
14 (2325, 3)
15 (3171, 3)
16 (56339, 3)
17 (13657, 3)
18 (4903, 3)
24 (2942, 3)
26 (1506, 3)
28 (11230, 3)
30 (107, 3)
31 (3380, 3)
41 (608370, 3)
43 (14570, 3)
44 (1161, 3)
46 (32377, 3)
47 (155810, 3)
49 (20916, 3)
50 (10663, 3)
51 (16384, 3)
52 (4407, 3)
53 (12918, 3)
54 (5575, 3)
58 (1520, 3)
60 (11769, 3)
62 (120, 3)
63 (4625, 3)
77 (3546, 3)
80 (43, 3)
85 (567, 3)
251 (2485, 3)
252 (1415, 3)
253 (1800, 3)
254 (1462, 3)
255 (2350, 3)
1000 (885, 3)
1001 (7105, 3)
1002 (5295, 3)
1003 (19937, 3)
1005 (10997, 3)
1006 (5123, 3)
1007 (28742, 3)
1008 (31987, 3)
1009 (34859, 3)
1010 (6365, 3)
1011 (32168, 3)
1012 (25684, 3)
1013 (19530, 3)
1014 (16325, 3)
1015 (29785, 3)
1016 (6085, 3)
1017 (10745, 3)
1018 (18995, 3)
1019 (6785, 3)
1020 (9564, 3)
1021 (8323, 3)
1022 (29368, 3)
1023 (8715, 3)
1024 (35496, 3)
1025 (29076, 3)
1026 (8206, 3)
1027 (53106, 3)
1

In [562]:
a=np.array([[1,2,3],[4,5,1]])
b=np.array([[1,1,1],[1,1,1]])
c=a==b
c

array([[ True, False, False],
       [False, False,  True]], dtype=bool)

# Connectivity between regions

So far the above only concerns the absolute number of endpoints belonging to the regions. However I wish to conduct each streamline based study.

In [212]:
import HTracking1_tools as HT

In [213]:
HT.test()

success


# Intermediary array vs list speedtest

In [55]:
some_list=[]
some_array=np.zeros(int(1e8))
some_array

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [56]:
begin=time.time()
some_array[:]=1
end=time.time()
print(end-begin)

0.24418139457702637


In [57]:
begin=time.time()
for i in np.arange(5e7):
    some_list.append(i)
end=time.time()
print(end-begin)

8.588871002197266


# map_voxel_to_ref construction framework

In [258]:
BS_label=labels==16
a=np.where(roi==True)
a

(array([105, 106, 107, ..., 153, 153, 153], dtype=int64),
 array([145, 145, 145, ..., 149, 149, 149], dtype=int64),
 array([93, 93, 91, ..., 63, 64, 65], dtype=int64))

In [426]:
asd=np.array([[1,2,3],[4,0,1]])
asdf=asd==1
np.where(asdf==True)

(array([0, 1], dtype=int64), array([0, 2], dtype=int64))

In [211]:
x,y,z=a
vec=np.zeros([4,x.shape[0]])
vec[0]=x
vec[1]=y
vec[2]=z

In [221]:
vecT=vec.T
vecT

array([[ 105.,  145.,   93.,    0.],
       [ 106.,  145.,   93.,    0.],
       [ 107.,  145.,   91.,    0.],
       ..., 
       [ 153.,  149.,   63.,    0.],
       [ 153.,  149.,   64.,    0.],
       [ 153.,  149.,   65.,    0.]])

In [228]:
np.dot(labels_img.affine,vec.T[0])

array([ -73.49999875,  101.49999827,   65.09999889,    0.        ])

In [234]:
np.dot(streamlines.affine,vecT[0])
vecT.shape[0]

56339

In [239]:
vecREF=np.zeros([x.shape[0],4])

In [240]:
for i in range(0,vecT.shape[0]):
    vecREF[i]=np.dot(streamlines.affine,vecT[i])

In [243]:
vecREF

array([[-131.25,  181.25,  116.25,    0.  ],
       [-132.5 ,  181.25,  116.25,    0.  ],
       [-133.75,  181.25,  113.75,    0.  ],
       ..., 
       [-191.25,  186.25,   78.75,    0.  ],
       [-191.25,  186.25,   80.  ,    0.  ],
       [-191.25,  186.25,   81.25,    0.  ]])

In [191]:
x.shape[0]

56339

In [148]:
b=np.array([[1,2],[3,4]])
print(b[0][1])

2


In [184]:
x.shape

(56339,)

In [139]:
mat=np.array([[1,0,2,0],[0,2,0,0],[0,0,3,0],[0,0,0,1]])
a=np.array([5,6,7,8])
np.dot(mat,a.T)

array([19, 12, 21,  8])

In [46]:
"""import matplotlib.pyplot as plt
plt.figure(figsize=[9,9])
plt.imshow(np.log1p(M[2000:2040,2000:2040]), interpolation='nearest')
#plt.imshow(M[0:60,0:60], interpolation='nearest')
plt.savefig("connectivity.png")
plt.show()"""

'import matplotlib.pyplot as plt\nplt.figure(figsize=[9,9])\nplt.imshow(np.log1p(M[2000:2040,2000:2040]), interpolation=\'nearest\')\n#plt.imshow(M[0:60,0:60], interpolation=\'nearest\')\nplt.savefig("connectivity.png")\nplt.show()'

In [28]:
asdf=[0.0, 2015.0, 2030.0, 2001.0, 2009.0, 41.0, 2031.0, 2022.0, 2008.0, 2024.0, 2018.0, 2020.0, 2011.0, 47.0, 2007.0, 2027.0, 2034.0, 2003.0, 2019.0, 2012.0, 46.0, 2035.0, 2029.0, 2033.0, 2013.0, 53.0, 44.0, 63.0, 2016.0, 2006.0, 51.0, 54.0, 2000.0, 43.0, 77.0, 2028.0, 2025.0, 60.0, 52.0, 62.0, 2021.0, 49.0, 2005.0, 50.0, 2010.0, 16.0, 2017.0, 80.0, 2014.0, 2032.0, 2023.0, 58.0, 2026.0, 2002.0, 15.0, 24.0, 85.0, 1013.0, 1005.0, 1025.0, 1029.0, 1026.0, 1028.0, 2.0, 1021.0, 8.0, 251.0, 1010.0, 1023.0, 1017.0, 14.0, 1014.0, 1002.0, 1024.0, 252.0, 28.0, 253.0, 254.0, 1000.0, 255.0, 1032.0, 7.0, 31.0, 1022.0, 10.0, 1012.0, 4.0, 1011.0, 26.0, 11.0, 1016.0, 17.0, 18.0, 1027.0, 13.0, 12.0, 1007.0, 1006.0, 5.0, 1033.0, 1008.0, 1003.0, 1035.0, 30.0, 1031.0, 1020.0, 1009.0, 1019.0, 1030.0, 1034.0, 1018.0, 1015.0, 1001.0]

In [29]:
asdf.sort()
for i in asdf:
    if i!=16:
        print(i)

0.0
2.0
4.0
5.0
7.0
8.0
10.0
11.0
12.0
13.0
14.0
15.0
17.0
18.0
24.0
26.0
28.0
30.0
31.0
41.0
43.0
44.0
46.0
47.0
49.0
50.0
51.0
52.0
53.0
54.0
58.0
60.0
62.0
63.0
77.0
80.0
85.0
251.0
252.0
253.0
254.0
255.0
1000.0
1001.0
1002.0
1003.0
1005.0
1006.0
1007.0
1008.0
1009.0
1010.0
1011.0
1012.0
1013.0
1014.0
1015.0
1016.0
1017.0
1018.0
1019.0
1020.0
1021.0
1022.0
1023.0
1024.0
1025.0
1026.0
1027.0
1028.0
1029.0
1030.0
1031.0
1032.0
1033.0
1034.0
1035.0
2000.0
2001.0
2002.0
2003.0
2005.0
2006.0
2007.0
2008.0
2009.0
2010.0
2011.0
2012.0
2013.0
2014.0
2015.0
2016.0
2017.0
2018.0
2019.0
2020.0
2021.0
2022.0
2023.0
2024.0
2025.0
2026.0
2027.0
2028.0
2029.0
2030.0
2031.0
2032.0
2033.0
2034.0
2035.0


In [30]:
def NS_gen(i,j):
    """
    prints out lists of number of streamlines between i and j
    i: (int), the 'seed' region,
    j: (list). the targets"""
    for p in j:
        if int(p)>i:
            print("between brainstem and region",p," : ",len(grouping[i,int(p)]))
        elif int(p)<i:
            print("between brainstem and region",p," : ",len(grouping[int(p),i]))

In [31]:
NS_gen(16,asdf)

between brainstem and region 0.0  :  186
between brainstem and region 2.0  :  0
between brainstem and region 4.0  :  0
between brainstem and region 5.0  :  0
between brainstem and region 7.0  :  0
between brainstem and region 8.0  :  0
between brainstem and region 10.0  :  0
between brainstem and region 11.0  :  0
between brainstem and region 12.0  :  0
between brainstem and region 13.0  :  0
between brainstem and region 14.0  :  0
between brainstem and region 15.0  :  0
between brainstem and region 17.0  :  0
between brainstem and region 18.0  :  0
between brainstem and region 24.0  :  0
between brainstem and region 26.0  :  0
between brainstem and region 28.0  :  0
between brainstem and region 30.0  :  0
between brainstem and region 31.0  :  0
between brainstem and region 41.0  :  3
between brainstem and region 43.0  :  0
between brainstem and region 44.0  :  0
between brainstem and region 46.0  :  9
between brainstem and region 47.0  :  70
between brainstem and region 49.0  :  0
bet

In [32]:
NS_gen(0,asdf)

between brainstem and region 2.0  :  0
between brainstem and region 4.0  :  0
between brainstem and region 5.0  :  0
between brainstem and region 7.0  :  0
between brainstem and region 8.0  :  0
between brainstem and region 10.0  :  0
between brainstem and region 11.0  :  0
between brainstem and region 12.0  :  0
between brainstem and region 13.0  :  0
between brainstem and region 14.0  :  0
between brainstem and region 15.0  :  7
between brainstem and region 16.0  :  186
between brainstem and region 17.0  :  0
between brainstem and region 18.0  :  0
between brainstem and region 24.0  :  0
between brainstem and region 26.0  :  0
between brainstem and region 28.0  :  0
between brainstem and region 30.0  :  0
between brainstem and region 31.0  :  0
between brainstem and region 41.0  :  7479
between brainstem and region 43.0  :  0
between brainstem and region 44.0  :  0
between brainstem and region 46.0  :  28812
between brainstem and region 47.0  :  282149
between brainstem and region 49

# Save as trackvis file 

In [34]:
start=time.time()
save_trk("Prob_.trk", streamlines, data_affine,
         labels.shape)
end=time.time()
print("time taken: ",end-start," seconds")
print("time taken: ",(end-start)/60," minutess")

time taken:  21724.882600307465  seconds
time taken:  362.0813766717911  minutess


# ROI save

In [118]:
Brainstem=labels==16
nib.save(nib.Nifti1Image(Brainstem.astype(np.float32),labels_img.affine),'Brainstem.nii.gz')
lhThalamus=labels==10
nib.save(nib.Nifti1Image(lhThalamus.astype(np.float32),labels_img.affine),'lhThalamus.nii.gz')
rhThalamus=labels==49
nib.save(nib.Nifti1Image(rhThalamus.astype(np.float32),labels_img.affine),'rhThalamus.nii.gz')



In [121]:
lhCerebellum=labels==8
nib.save(nib.Nifti1Image(lhCerebellum.astype(np.float32),labels_img.affine),'lhCerebellum.nii.gz')
rhCerebellum=labels==47
nib.save(nib.Nifti1Image(rhCerebellum.astype(np.float32),labels_img.affine),'rhCerebellum.nii.gz')
rhCerebellum=labels==47
nib.save(nib.Nifti1Image(rhCerebellum.astype(np.float32),labels_img.affine),'rhCerebellum.nii.gz')
rhCerebellum=labels==47
nib.save(nib.Nifti1Image(rhCerebellum.astype(np.float32),labels_img.affine),'rhCerebellum.nii.gz')

In [8]:
from dipy.reconst.shm import CsaOdfModel

In [9]:
csa_model = CsaOdfModel(gtab, sh_order=6)

In [59]:
gfa = csa_model.fit(data[:,:,70:90], mask=mask_bool[:,:,70:90]).gfa

C:\Users\David\Miniconda3\lib\site-packages\dipy\reconst\shm.py:731: RuntimeWarning: Mean of empty slice.
  b0 = out[..., where_b0].mean(-1)
C:\Users\David\Miniconda3\lib\site-packages\numpy\core\_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [60]:
gfa.shape

(145, 174, 20)

In [61]:
dist=[]

for i in range(0,gfa.shape[0]):
    for j in range (0,gfa.shape[1]):
        for k in range(0,gfa.shape[2]):
            if not np.isnan(gfa[i][j][k]) and gfa[i][j][k] not in dist:
                dist.append(gfa[i][j][k])
print(dist)

[]


In [51]:
not np.isnan(gfa[1][2][3])

False

In [39]:
if 1 not in gfa:
    print(0)
    

0


In [ ]:
print("Done")

In [70]:
print(Atlas_labels_img)
print(labels_img)

<class 'nibabel.nifti1.Nifti1Image'>
data shape (91, 109, 91)
affine: 
[[  -2.    0.    0.   90.]
 [   0.    2.    0. -126.]
 [   0.    0.    2.  -72.]
 [   0.    0.    0.    1.]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3  91 109  91   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [-1.  2.  2.  2.  1.  1.  1.  1.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 50
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : scanner
sform_code    

In [22]:
from dipy.data import read_stanford_labels
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel
from dipy.tracking import utils
from dipy.tracking.local import (ThresholdTissueClassifier, LocalTracking)

seed_mask = mm == 1
white_matter = (mm == 1)
seeds = utils.seeds_from_mask(seed_mask, density=1, affine=affine)

csd_model = ConstrainedSphericalDeconvModel(gtab, None, sh_order=6)
csd_fit = csd_model.fit(data, mask=white_matter)

KeyboardInterrupt: 

In [ ]:
csa_model = CsaOdfModel(gtab, sh_order=6)
gfa = csa_model.fit(data, mask=white_matter).gfa
classifier = ThresholdTissueClassifier(gfa, .25)

In [36]:
import dipy
gtab=dipy.core.gradients.gradient_table(bvals=bvals,bvecs=bvecs)

In [37]:
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel
from dipy.tracking import utils
from dipy.tracking.local import (ThresholdTissueClassifier, LocalTracking)

seed_mask = (mmm == 1)
white_matter = (mmm == 1)
seeds = utils.seeds_from_mask(seed_mask, density=1, affine=affine)

csd_model = ConstrainedSphericalDeconvModel(gtab, None, sh_order=6)
csd_fit = csd_model.fit(data, mask=white_matter)

In [45]:
print(bvals.shape)
print(bvecs.shape)

(288,)
(288, 3)


In [50]:
csamodel = shm.CsaOdfModel(gtab, 6)
csapeaks = peaks.peaks_from_model(model=csamodel,
                                  data=data,
                                  sphere=peaks.default_sphere,
                                  relative_peak_threshold=.8,
                                  min_separation_angle=45,
                                  mask=mmm)

C:\Users\David\Miniconda3\lib\site-packages\dipy\reconst\shm.py:731: RuntimeWarning: Mean of empty slice.
  b0 = out[..., where_b0].mean(-1)
C:\Users\David\Miniconda3\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


ValueError: odf can not have nans

In [266]:
SOLAR_MASS=1
DAYS_PER_YEAR=365.25

In [284]:
BODIES = {
        'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

        'jupiter': ([4.84143144246472090e+00,
            -1.16032004402742839e+00,
            -1.03622044471123109e-01],
            [1.66007664274403694e-03 * DAYS_PER_YEAR,
                7.69901118419740425e-03 * DAYS_PER_YEAR,
                -6.90460016972063023e-05 * DAYS_PER_YEAR],
            9.54791938424326609e-04 * SOLAR_MASS),

        'saturn': ([8.34336671824457987e+00,
            4.12479856412430479e+00,
            -4.03523417114321381e-01],
            [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
            2.85885980666130812e-04 * SOLAR_MASS),

        'uranus': ([1.28943695621391310e+01,
            -1.51111514016986312e+01,
            -2.23307578892655734e-01],
            [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
            4.36624404335156298e-05 * SOLAR_MASS),

        'neptune': ([1.53796971148509165e+01,
            -2.59193146099879641e+01,
            1.79258772950371181e-01],
            [2.68067772490389322e-03 * DAYS_PER_YEAR,
                1.62824170038242295e-03 * DAYS_PER_YEAR,
                -9.51592254519715870e-05 * DAYS_PER_YEAR],
            5.15138902046611451e-05 * SOLAR_MASS) 
        }
a=list(BODIES.values())
a

[([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], 1),
 ([8.34336671824458, 4.124798564124305, -0.4035234171143214],
  [-1.010802020429865, 1.8257123565105353, 0.008415991793881727],
  0.0002858859806661308),
 ([4.841431442464721, -1.1603200440274284, -0.10362204447112311],
  [0.6063429937622595, 2.812063835028102, -0.025219052119904602],
  0.0009547919384243266),
 ([15.379697114850917, -25.919314609987964, 0.17925877295037118],
  [0.979117539021147, 0.59471528106468, -0.03475690709633262],
  5.1513890204661145e-05),
 ([12.894369562139131, -15.111151401698631, -0.22330757889265573],
  [1.082820652455292, 0.8687368028870042, -0.010832933990932176],
  4.366244043351563e-05)]

In [283]:
def combinations(l):
    result = []
    for x in range(len(l) - 1):
        ls = l[x+1:]
        for y in ls:
            result.append((l[x],y))
    return result
combinations(a)

[(([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], 1),
  ([8.34336671824458, 4.124798564124305, -0.4035234171143214],
   [-1.010802020429865, 1.8257123565105353, 0.008415991793881727],
   0.0002858859806661308)),
 (([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], 1),
  ([4.841431442464721, -1.1603200440274284, -0.10362204447112311],
   [0.6063429937622595, 2.812063835028102, -0.025219052119904602],
   0.0009547919384243266)),
 (([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], 1),
  ([15.379697114850917, -25.919314609987964, 0.17925877295037118],
   [0.979117539021147, 0.59471528106468, -0.03475690709633262],
   5.1513890204661145e-05)),
 (([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], 1),
  ([12.894369562139131, -15.111151401698631, -0.22330757889265573],
   [1.082820652455292, 0.8687368028870042, -0.010832933990932176],
   4.366244043351563e-05)),
 (([8.34336671824458, 4.124798564124305, -0.4035234171143214],
   [-1.010802020429865, 1.8257123565105353, 0.008415991793881727],
   0.0002858859806661308),
  ([4.841431442464721, -1.1603200440

In [298]:
a=enumerate(list(BODIES.values()))
a

In [297]:
for i,body in a:
    print(body)

In [65]:
del sys.modules['Proxy']

In [66]:
import Proxy as prox

8


In [67]:
ob=prox.out()
ob.p

9

In [60]:
prob_dg

In [367]:
abc=labels==16

In [368]:
abc

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ..., 
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ..., 
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ..., 
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, 

In [372]:
abcd=np.where(abc)

In [373]:
abcd

(array([105, 106, 107, ..., 153, 153, 153], dtype=int64),
 array([145, 145, 145, ..., 149, 149, 149], dtype=int64),
 array([93, 93, 91, ..., 63, 64, 65], dtype=int64))

In [374]:
abcde=np.array(abcd)

In [375]:
abcde

array([[105, 106, 107, ..., 153, 153, 153],
       [145, 145, 145, ..., 149, 149, 149],
       [ 93,  93,  91, ...,  63,  64,  65]], dtype=int64)

In [376]:
abcde.T

array([[105, 145,  93],
       [106, 145,  93],
       [107, 145,  91],
       ..., 
       [153, 149,  63],
       [153, 149,  64],
       [153, 149,  65]], dtype=int64)

# read annot files

In [179]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import os
import time
import pickle
import sys
import random
import collections

In [180]:
directory="C:\\Users\\David\\Desktop\\host\\104012_3T_Structural_preproc\\104012\\T1w"

lh_BA_th=open(directory+"\\104012\\stats\\lh.BA.thresh.stats",'r')
lh_aparc_stats=open(directory+"\\104012\\stats\\lh.aparc.stats",'r')

In [181]:
Master=[]

for line in lh_aparc_stats:
    li=line.rstrip()
    if not li.startswith("#") or li.startswith("# ColHeaders"):
        Master.append(li.split())
    

In [182]:
Master[0].pop(0)
Master[0].pop(0)

'ColHeaders'

BA 1 - somatosensory area\\
BA 2 - somatosensory area\\
BA 3a - somatosensory area\\
BA 3b - somatosensory area\\
BA 4a - primary motor area (anterior)\\
BA 4p - primary motor area (posterier)\\
BA 6 - pre-motor area\\
BA 44 - Broca's area (pars opercularis)\\
BA 45 - Broca's area (pars triangularis)\\
V1 (BA 17) - primary visual area\\
V2 (BA 18) - secondary visual area\\
V5/MT - visual area, middle temporal\\

In [183]:
Primary_sum=0
for i in range(1,len(Master)):
    Primary_sum+=int(Master[i][3])
Primary_sum

247943

In [184]:
lh_BA_th=open(directory+"\\104012\\stats\\lh.BA.thresh.stats",'r')

In [185]:
Master=[]

for line in lh_BA_th:
    li=line.rstrip()
    if not li.startswith("#") or li.startswith("# ColHeaders"):
        Master.append(li.split())
    

In [186]:
Master[0].pop(0)
Master[0].pop(0)

'ColHeaders'

In [187]:
select=[1,2,3,4,5,6,10]

Primary_sum=0
for i in select:
    Primary_sum+=int(Master[i][3])
Primary_sum

17249

In [188]:
Primary_sum+=1076

In [190]:
247943/Primary_sum

13.53031377899045